In [1]:
from koebe.geometries.spherical2 import PointS2, DiskS2, CircleArcS2
from koebe.geometries.euclidean3 import PointE3
from koebe.geometries.orientedProjective2 import DiskOP2



# Step 1: Generate a random convex hull

In [2]:
from koebe.algorithms.incrementalConvexHull import randomConvexHullE3WithHighDegreeVertex

mesh = randomConvexHullE3WithHighDegreeVertex(100, 20)
mesh.outerFace = mesh.verts[-1].remove()

In [3]:
# Run this block if you want to see the mesh
from koebe.graphics.spherical2viewer import *
viewer = S2Viewer(600,600)
viewer.toggleSphere() # Hide the sphere
viewer.addAll(mesh.edges)
viewer.show()

<IPython.core.display.Javascript object>

S2Sketch(height=600, objects='[{"type": "SegmentE3", "endpoints": [[-0.3785859973939298, 0.1363018926277303, 0…

# Step 2: Circle pack the convex hull

In [4]:
from koebe.geometries.euclidean2 import PointE2
from koebe.algorithms.hypPacker import *

def orthProj(p):
    return PointE2(p.x, p.y)

dists = [(orthProj(v.data) - PointE2.O).normSq() for v in mesh.verts]
closestToOriginIdx = dists.index(min(dists))
packing, _ = maximalPacking(
    mesh, 
    num_passes=1000, 
    centerDartIdx = mesh.darts.index(mesh.verts[closestToOriginIdx].aDart)
)

In [5]:
# Run this to view the circle packing
from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
viewer = PoincareDiskViewer(600, 600)
viewer.addAll(packing.verts)
#d = DiskOP2(1,0,0,-1)
#viewer.add(d)
#viewer.setStyle(d, makeStyle(fill='#00ff00'))
viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=600, objects='[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#000…

In [6]:
from koebe.algorithms.inversiveVoronoi import inversiveVoronoi as IV
from koebe.geometries.orientedProjective2 import DiskOP2


sgPacking = packing.duplicate(
    vdata_transform = lambda vData : DiskOP2.fromCircleE2(vData.toPoincareCircleE2()).toDiskS2()
)



In [7]:
# Run this block if you want to see the mesh
from koebe.graphics.spherical2viewer import *
viewer = S2Viewer(600,600)
viewer.toggleSphere() # Hide the sphere
viewer.addAll(sgPacking.verts)
blueStyle = makeStyle(stroke="#00f", strokeWeight=1, fill="rgba(255, 255, 255, 0.5)")
for v in sgPacking.verts:
    viewer.setStyle(v, blueStyle)
viewer.show()

<IPython.core.display.Javascript object>

S2Sketch(height=600, objects='[{"type": "DiskS2", "disk": [0.5791318409015689, -1.3141506503241862, -0.2702062…

In [12]:
arcs = IV(sgPacking)

arcs2D = [arc.sgToCircleArcOP2() for arc in arcs]

disks2D = [v.data.sgProjectToOP2() for v in sgPacking.verts]


In [15]:
# Get toSVG method implemented here. 
arcs2D[0]

CircleArcOP2(source=PointOP2(hx=0.27923230343280064, hy=-0.8924014144518542, hw=1.354470078009333), target=PointOP2(hx=0.9599877279250455, hy=-0.2772347875964489, hw=0.9604470635772674), disk=DiskOP2(a=-4.516415004205652, b=2.914365230525287, c=1.1446577785040664, d=2.3058392948545725))

In [ ]:
# Something along these lines

print "Writing SVG file..."
svgStr = toSVG(512.0)
f = open(svgFilePath, 'w')
f.write(svgStr)
f.close()
print "Done."